# Daily Traffic Counts
Questions:
- Can we get time stamp of the image?
- How can we get live speed data?

## Imports

In [360]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import torch
import random
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt
import segmentation_models_pytorch as smp
import albumentations as album
from PIL import Image
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import random_split
from sklearn.metrics import accuracy_score
import plotly.express as px
import torchmetrics
from torchmetrics import MeanAbsolutePercentageError


%matplotlib inline

## Global Variables

In [361]:
IMG_SIZE = 1024
VEHICLE_DETECTION_COUNT_PATH = '/home/ah2719/FYP/Spatial_Finance_Transport/data/vehicle_counts_detection.csv'
DAILY_COUNT_PATH = '/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/report_daily_count.csv'
SPEED_DATA_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/avg_mph.csv"
ROAD_WIDTH_PATH = ""
TIMESTAMP_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/timestamp.csv"
AVG_MPH_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/avg_mph.csv"

NN_MODEL_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/models/nn_model.pth"

## True Count Data
- Traffic monitoring stations for long-term traffic count data
    - Extract at same time as Satellite Image!
- How to use permanent and temporary traffic count stations

In [362]:
# Get Ground Truth Data

## Vehicle detection number
From vehicle detection model

## Road characteristics
From road characterstics pipeline

Includes:
- Road width
- Live speed data
- Directionality

## Neural Network Model

In [363]:
class CustomDataset(Dataset):
    def __init__(self):
        self.labels = torch.tensor(pd.read_csv(DAILY_COUNT_PATH)['daily_count'].values.astype('float32'))
        self.vehicle_count = torch.tensor(pd.read_csv(DAILY_COUNT_PATH)['total_volume_normalised'].values.astype('float32')).unsqueeze(1) # Training
        #self.speed_data = pd.read_csv(SPEED_DATA_PATH) 
        #self.road_width = pd.read_csv(ROAD_WIDTH_PATH)
        self.hour = torch.tensor(pd.read_csv(DAILY_COUNT_PATH)['hour'].values.astype('float32')).unsqueeze(1)
        self.avg_mph = torch.tensor(pd.read_csv(DAILY_COUNT_PATH)['avg_mph'].values.astype('float32')).unsqueeze(1)
        self.day = torch.tensor(pd.read_csv(DAILY_COUNT_PATH)['day'].values.astype('float32')).unsqueeze(1)
        self.month = torch.tensor(pd.read_csv(DAILY_COUNT_PATH)['month'].values.astype('float32')).unsqueeze(1)

        self.x = torch.concat((self.vehicle_count, self.avg_mph, self.day, self.month, self.hour), dim=-1)
        self.y = self.labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

custom_data = CustomDataset()
train_split = 0.8
train_data, test_data = random_split(custom_data, [train_split, 1-train_split])

In [364]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(5, 5),
            nn.Linear(5,20),
            nn.Linear(20,1)
            #nn.ReLU(),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [365]:
nn_model = NeuralNetwork()
nn_model

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=5, out_features=5, bias=True)
    (1): Linear(in_features=5, out_features=20, bias=True)
    (2): Linear(in_features=20, out_features=1, bias=True)
  )
)

In [366]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

nn_model.apply(init_weights)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=5, out_features=5, bias=True)
    (1): Linear(in_features=5, out_features=20, bias=True)
    (2): Linear(in_features=20, out_features=1, bias=True)
  )
)

In [367]:
learning_rate = 1e-1
batch_size = 1
epochs = 5
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(nn_model.parameters(), lr=learning_rate)

MAPE = MeanAbsolutePercentageError()

### DataLoaders

In [368]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False, sampler=None,
                    batch_sampler=None, num_workers=0, collate_fn=None,
                    pin_memory=False, drop_last=False, timeout=0,
                    worker_init_fn=None, prefetch_factor=2,
                    persistent_workers=False)

In [369]:
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, sampler=None,
                    batch_sampler=None, num_workers=0, collate_fn=None,
                    pin_memory=False, drop_last=False, timeout=0,
                    worker_init_fn=None, prefetch_factor=2,
                    persistent_workers=False)

### Training & Testing

In [370]:
def run_epoch(ep_id, action, loader, model, optimizer, criterion):
    losses = [] # Keep list of accuracies to track progress
    is_training = action == "train" # True when action == "train", else False 

    # Looping over all batches
    for batch_idx, batch in enumerate(loader): 
            x, y = batch
            #print("x: {}".format(x))
            #print("y: {}".format(y))

            # Resetting the optimizer gradients
            optimizer.zero_grad()

            # Setting model to train or test
            with torch.set_grad_enabled(is_training):
                
                # Feed batch to model
                logits = nn_model(x).squeeze(1)
                print("logits: {}".format(logits))

                # Calculate the loss based on predictions and real labels
                loss = criterion(logits, y)
                print("loss: {}".format(loss))
                mape_loss = MAPE(logits, y)

                # If training, perform backprop and update weights
                if is_training:
                    loss.backward()
                    optimizer.step()

                # Append current batch accuracy
                losses.append(mape_loss.detach().numpy())

                # Print some stats every 50th batch 
                if batch_idx % 50 == 0:
                    print(f"{action.capitalize()}ing, Epoch: {ep_id+1}, Batch {batch_idx}: Loss = {loss.item()}")
    # Return accuracies to main loop                 
    return losses

In [371]:
def main(epochs, train_dl, test_dl, model, optimizer, criterion):

    # Keep lists of accuracies to track performance on train and test sets
    train_losses = []
    test_losses = []

    # Looping over epochs
    for epoch in range(epochs):
        
        # Looping over train set and training
        train_loss = run_epoch(epoch, "train", train_dl, model, optimizer, criterion)

        # Looping over test set
        test_loss = run_epoch(epoch, "test", test_dl, model, optimizer, criterion) 

        # Collecting stats
        train_losses += train_loss
        test_losses += test_loss         
            
    return train_losses, test_losses

In [372]:
train_losses, test_losses = main(epochs=epochs, train_dl=train_dataloader, test_dl=test_dataloader, model=nn_model, optimizer=optimizer, criterion=loss_fn)

logits: tensor([14.9474], grad_fn=<SqueezeBackward1>)
loss: 268207856.0
Training, Epoch: 1, Batch 0: Loss = 268207856.0
logits: tensor([143.6878], grad_fn=<SqueezeBackward1>)
loss: 264007664.0
logits: tensor([404.1245], grad_fn=<SqueezeBackward1>)
loss: 255612176.0
logits: tensor([602.8307], grad_fn=<SqueezeBackward1>)
loss: 249297856.0
logits: tensor([1366.7476], grad_fn=<SqueezeBackward1>)
loss: 225758192.0
logits: tensor([2151.1670], grad_fn=<SqueezeBackward1>)
loss: 202801328.0
logits: tensor([3280.4517], grad_fn=<SqueezeBackward1>)
loss: 171912720.0
logits: tensor([4516.3096], grad_fn=<SqueezeBackward1>)
loss: 141032016.0
logits: tensor([5981.1445], grad_fn=<SqueezeBackward1>)
loss: 108385912.0
logits: tensor([8891.6953], grad_fn=<SqueezeBackward1>)
loss: 56254572.0
logits: tensor([10147.3818], grad_fn=<SqueezeBackward1>)
loss: 38995256.0
logits: tensor([12017.7373], grad_fn=<SqueezeBackward1>)
loss: 19134174.0
logits: tensor([19516.6113], grad_fn=<SqueezeBackward1>)
loss: 9763196

### Loss Curve Plot

In [373]:
px.line(train_losses[50:])

In [374]:
px.line(test_losses[50:])

### Random test sample

In [375]:
random_idx = np.random.randint(0,len(test_data))
x, y = test_data[random_idx]
print("x: {}".format(x))
print("y: {}".format(y))
pred_y = nn_model(x)
print("pred_y: {}".format(pred_y))

x: tensor([ 0.3877, 68.0000,  1.0000,  6.0000, 22.0000])
y: 16392.0
pred_y: tensor([16427.3359], grad_fn=<AddBackward0>)


## Save Model

In [377]:
torch.save(nn_model, NN_MODEL_PATH)